<a href="https://colab.research.google.com/github/MattLeRoi/new_project/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bank

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df=pd.read_csv('bank-full.csv', delimiter=';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [10]:
df.y.value_counts()

y
no     39922
yes     5289
Name: count, dtype: int64

In [11]:
X=df.drop(['y','pdays'], axis=1) # The -1 in pdays throws off the math. previous also functions as a flag for previously contacted
y = [1 if target_y_n == "yes" else 0 for target_y_n in df['y']]

In [12]:
categorical_features = ['job','marital','education','default','housing','loan','contact','month','poutcome']

X_encoded = pd.get_dummies(X, columns=categorical_features)
X_encoded

,age,balance,day,duration,campaign,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,44,29,5,151,1,0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,33,2,5,76,1,0,False,False,True,False,...,False,False,True,False,False,False,False,False,False,True
3,47,1506,5,92,1,0,False,True,False,False,...,False,False,True,False,False,False,False,False,False,True
4,33,1,5,198,1,0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,17,977,3,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
45207,71,1729,17,456,2,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
45208,72,5715,17,1127,5,3,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
45209,57,668,17,508,4,0,False,True,False,False,...,False,False,False,True,False,False,False,False,False,True


In [13]:

# # Convert to pandas categoricals (needed for automatic handling)
# for col in categorical_features:
#     df[col] = df[col].astype("category").cat.codes

X_all_training,X_test,y_all_training,y_test = train_test_split(X,y,random_state=42, test_size=.15) # 15% test set
X_train,X_val,y_train,y_val = train_test_split(X_all_training,y_all_training,random_state=42, test_size=.1/.85) # 10% validation set
X_val

NameError: name 'train_test_split' is not defined

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
assert isinstance(X, pd.DataFrame)   
model = HistGradientBoostingClassifier()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, y_pred)
print("ROC AUC Score:", roc_auc)

In [ ]:
categorical_features = ['job','marital','education','default','housing','loan','contact','month','poutcome']

# Convert to pandas categoricals (needed for automatic handling)
for col in categorical_features:
    bank_train_X[col] = bank_train_X[col].astype("category").cat.codes

    
X,X_val,y,y_val = train_test_split(bank_train_X,bank_train_y,random_state=42)
print(bank_train_X.dtypes)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
assert isinstance(X, pd.DataFrame)   
model = HistGradientBoostingClassifier()
model.fit(X, y)

# Predict
y_pred = model.predict(X_val)


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, y_pred)
print("ROC AUC Score:", roc_auc)

In [ ]:
X.campaign.value_counts()

In [ ]:
bfu = pd.read_csv('./bank/bank-full.csv', delimiter=';')

In [ ]:
bfu

In [ ]:
bank=pd.read_csv('./bank/bank.csv', delimiter=';')

In [ ]:
bfu.duplicated().sum()

In [ ]:
combined=pd.concat([bank,bfu])

In [ ]:
combined.duplicated().sum()

In [ ]:
result = bfu.assign(y_binary=(bfu['y'] == 'yes').astype(int)) \
            .groupby('day')['y_binary'].mean() * 100

In [ ]:
result

In [ ]:
bfu.day.value_counts()